In [ ]:
import pandas as pd


In [14]:
import sys
# Add the directory containing custom modules to the Python path
sys.path.append('../../common')

# Reloadable imports
import importlib

# Import the support2 module
import support2
importlib.reload(support2)

# Import specific functions from support2
from support2 import (
    viewStats, 
    printAllUniqueValueCount, 
    compareTestTrain, 
    compareValuesInColumns, 
    GetColumnsWithTypeObject, 
    combineColumns, 
    clean_feature_names
)

# Import ml_utils module and alias it as mt
# import ml_utils as mt
# importlib.reload(mt)

In [9]:
df = pd.read_csv('../data/stacksample/Questions.csv', encoding='latin1')
df_tags = pd.read_csv('../data/stacksample/tags.csv', encoding='latin1')
df_answers = pd.read_csv('../data/stacksample/Answers.csv', encoding='latin1')

In [3]:
df.dtypes

Id                int64
OwnerUserId     float64
CreationDate     object
ClosedDate       object
Score             int64
Title            object
Body             object
dtype: object

Download a small sample:


In [ ]:
# df.head(30).to_csv("questionsSample.csv")
# df_answers.head(30).to_csv("answers.csv")
# df_tags.head(30).to_csv("tagsSample.csv")

In [15]:
# viewStats(cd_train, True,"stats_cd_train_no_index.csv")
viewStats(df)

,Total Unique Values,Total Missing Values,missing %,Non-Missing Values,Total Values in Column,Data Type
Id,1264216,0,0.0,1264216,1264216,int64
OwnerUserId,630909,14454,1.1,1249762,1264216,float64
CreationDate,1264207,0,0.0,1264216,1264216,object
ClosedDate,55949,1208257,95.6,55959,1264216,object
Score,532,0,0.0,1264216,1264216,int64
Title,1263995,0,0.0,1264216,1264216,object
Body,1264204,0,0.0,1264216,1264216,object


In [16]:
viewStats(df_tags)

,Total Unique Values,Total Missing Values,missing %,Non-Missing Values,Total Values in Column,Data Type
Id,1264216,0,0.0,3750994,3750994,int64
Tag,37034,1113,0.0,3749881,3750994,object


In [17]:
viewStats(df_answers)

,Total Unique Values,Total Missing Values,missing %,Non-Missing Values,Total Values in Column,Data Type
Id,2014516,0,0.0,2014516,2014516,int64
OwnerUserId,468797,13200,0.7,2001316,2014516,float64
CreationDate,2002766,0,0.0,2014516,2014516,object
ParentId,1102568,0,0.0,2014516,2014516,int64
Score,697,0,0.0,2014516,2014516,int64
Body,2014375,0,0.0,2014516,2014516,object


🔍 Project Goal:

Given the text of a question (Title + Body), predict the correct Tags.

That’s the core task — it’s a multi-class, multi-label text classification problem, where the input is the question text, and the output is one or more tags from the top 10 most common.

For this:
I don’t need:

- Answers
- User info
- Post dates
- Scores

Will just keep:

- Title
- Body
- Tags ✅


In [21]:
columnsToDelete =['CreationDate','OwnerUserId','ClosedDate','Score']
df.drop(columns=columnsToDelete, inplace=True)

In [24]:
df.columns

Index(['Id', 'Title', 'Body'], dtype='object')

Now tags is repeating ids once for each tag the id has, considering each id can have multiple tags


In [22]:
df_tags.shape

(3750994, 2)

In [25]:
# Group tags by question Id
df_tags_grouped = df_tags.groupby('Id')['Tag'].apply(list).reset_index(name='Tags')

In [27]:
print(df_tags_grouped.shape); print(df.shape)

(1264216, 2)
(1264216, 3)


In [28]:
compareValuesInColumns(df, df_tags, columnName=["Id"])

The columns have the same values and order.


In [29]:
df_merged = pd.merge(df, df_tags_grouped, on='Id')

In [31]:
print(df_merged.shape)
print(df_merged.columns)

(1264216, 4)
Index(['Id', 'Title', 'Body', 'Tags'], dtype='object')


In [32]:
df_merged.dtypes

Id        int64
Title    object
Body     object
Tags     object
dtype: object

In [ ]:
# Merge questions with their tags
df_merged.sample(100).to_csv('df_merged_sample.csv')

In [34]:
# Optional: Keep only what you need
df_final = df_merged[['Title', 'Body', 'Tags']]

In [35]:
df_final.to_csv("final_merged_data.csv")